In [ ]:
# Extract sift feature from the image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
import random

num_of_centroids=4096
num_train_samples=300
num_test_samples=100

all_image_idx=random.sample(range(0,400),400)

def remove_noise (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(gray, 3)
    sharpen = cv2.Canny(median, 100,250)
    return sharpen


def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


In [ ]:


all_des=np.empty((1,128))

#SIFT
sift = cv2.SIFT_create()

num_of_desc=[]

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 1 image"+str(i))


num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FIRST SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 2 image"+str(i))

num_of_desc+=[all_des.shape[0]]

print("FINISHED READING SECOND SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Marhey/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 3 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING THIRD SET OF IMAGES")


for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 4 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FOURTH SET OF IMAGES")

desc_labels=np.zeros(all_des.shape[0])
desc_labels[num_of_desc[0]:num_of_desc[1]]=1
desc_labels[num_of_desc[1]+1:num_of_desc[2]]=2
desc_labels[num_of_desc[2]+1:num_of_desc[3]]=3


In [ ]:
# different options for kmeans
# - mini batches
# - limiting the number of descriptors
kmeans=MiniBatchKMeans(n_clusters=num_of_centroids,batch_size=num_of_centroids*1024,max_iter=20).fit(X=all_des)


In [ ]:
print(kmeans.cluster_centers_.shape)
print(kmeans.labels_.shape)
print(desc_labels.shape)

In [ ]:
centroids_labels=np.zeros((num_of_centroids,4))
for i in range(num_of_centroids):
    centroids_labels[i][0]=(np.sum(desc_labels[kmeans.labels_==i]==0))
    centroids_labels[i][1]=(np.sum(desc_labels[kmeans.labels_==i]==1))
    centroids_labels[i][2]=(np.sum(desc_labels[kmeans.labels_==i]==2))
    centroids_labels[i][3]=(np.sum(desc_labels[kmeans.labels_==i]==3))
    centroids_labels[i]/=np.sum(centroids_labels[i])
    
    
    
with open('kmeans_centers.txt', 'wb') as f:
    np.save(f, kmeans.cluster_centers_)


with open('kmeans_labels.txt', 'wb') as f:
    np.save(f, centroids_labels)


In [ ]:


def predict(path,centroids,centroids_labels):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            idx=kmeans.predict([des])
            dist=np.linalg.norm(des-centroids[i])
            predections+=(centroids_labels[idx]/dist)
        return np.argmax(predections)
    else:
        return -1

num_right0=0
num_right1=0
num_right2=0
num_right3=0

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/IBM Plex Sans Arabic/"+rand_idx+".jpeg"
      
    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type0: ",predicted)
    num_right0+=predicted==0

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/Lemonada/"+rand_idx+".jpeg"
    
    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type1: ",predicted)
    num_right1+=predicted==1
    
for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Marhey/"+rand_idx+".jpeg"
      
    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type2: ",predicted)
    num_right2+=predicted==2
    
for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Scheherazade New/"+rand_idx+".jpeg"
      
    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type3: ",predicted)
    num_right3+=predicted==3


In [ ]:
print(num_right0)
print(num_right1)
print(num_right2)
print(num_right3)

In [ ]:
(num_right0+num_right1+num_right2+num_right3)/400

In [ ]:
img=cv2.imread("../../fonts-dataset/Marhey/233.jpeg")
removed_noise=remove_noise(img)
_ , descriptors= sift.detectAndCompute(removed_noise,None)

predections=[0.0,0.0,0.0,0.0]
for des in descriptors:
    idx=kmeans.predict([des])
    dist=np.linalg.norm(des-kmeans.cluster_centers_[i])
    predections+=(centroids_labels[idx]/dist)
print(predections)

show_images([removed_noise])


In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
classes=np.arange(0, num_of_centroids, 1, dtype=int)
knn.fit(kmeans.cluster_centers_,classes)
_ , descriptors= sift.detectAndCompute(removed_noise,None)

predections=[0.0,0.0,0.0,0.0]
for des in descriptors:
    idx=knn.predict([des])
    dist=np.linalg.norm(des-kmeans.cluster_centers_[i])
    predections+=(centroids_labels[idx]/dist)
print(predections)